In [1]:
!pip install selenium
!apt update
!apt install chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics

In [2]:
"""
  1. Get all teams in the nfl (2013-2023)
  2. Get all urls for each team in the nfl
  3. Get the yearly roster for each team
    3.1 Get Starters Table
    3.2 Get Full Roster Table
"""

'\n  1. Get all teams in the nfl (2013-2023)\n  2. Get all urls for each team in the nfl\n  3. Get the yearly roster for each team\n    3.1 Get Starters Table\n    3.2 Get Full Roster Table\n'

# Importing Necessary Libraries

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import selenium
import string
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import json
import re
from selenium import webdriver
from multiprocessing import Pool
import datetime
import fcntl
from pytz import timezone
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Helper Functions

In [5]:
def get_webdriver():
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(options=options)

  # set implicit wait time
  driver.implicitly_wait(3) # seconds

  return driver

def transform_col(x):
  x = x.lower().replace(" ", "_").replace("%", "_pct").replace("-", "_")
  return x

def get_current_team_name(name):
  if name in ["Washington Redskins", "Washington Football Team"]:
    return "Washington Commanders"
  elif name in ["Oakland Raiders"]:
    return "Las Vegas Raiders"
  elif name in ["San Diego Chargers"]:
    return "Los Angeles Chargers"
  elif name in ["St. Louis Rams"]:
    return "Los Angeles Rams"
  else:
    return name

def transform_dataframe(df, year_cutoff=2013):
  column_name_map = {
    'tm': 'team_nm',
    'from': 'first_year',
    'to': 'last_year',
    'w': 'games_won',
    'l': 'games_lost',
    't': 'games_tied',
    'w_l_pct': 'win_loss_pct',
    'yr_plyf': 'years_in_playoffs',
    'playoffs_w_plyf': 'playoff_wins',
    'playoffs_l_plyf': 'playoff_losses',
    'playoffs_w_l_pct': 'playoff_win_loss_pct',
    'sbwl': "super_bowls_wins",
    'conf': "conf_champ_wins",
    'div': "div_champ_wins",
  }

  column_type_map = {
    'team_nm': 'object',
    'first_year': 'float',
    'last_year': 'float',
    'games_won': 'float',
    'games_lost': 'float',
    'games_tied': 'float',
    'win_loss_pct': 'float',
    'years_in_playoffs': 'float',
    'playoff_wins': 'float',
    'playoff_losses': 'float',
    'playoff_win_loss_pct': 'float',
    'super_bowls_wins': "float",
    'conf_champ_wins': "float",
    'div_champ_wins': "float",
  }

  delete_indices = []
  for ind in df.index:
    try:
      row_list = [transform_col(i) for i in df.loc[ind, :].values.flatten().tolist()[:6]]
      if row_list == ['tm', 'from', 'to', 'w', 'l', 't']:
        delete_indices.append(ind)
    except:
      continue

  df = df.drop(delete_indices, axis=0)

  df = df.drop([i for i in df.columns if i not in column_name_map.keys()], axis=1)

  df = df.rename(columns=column_name_map)

  df = df.dropna(subset=['team_nm'])

  df = df.astype(column_type_map)

  # filter teams by there end year (must be on/after 2013)
  df = df[df['last_year'] >= year_cutoff]

  # dealing with "duplicate" summary rows
  df_list = []
  for _, g in df.groupby("team_nm"):
    if len(g) > 1:
      drop_idx = g['first_year'].idxmin()
      g = g.drop(drop_idx, axis=0)
      df_list.append(g)
    else:
      df_list.append(g)

  df = pd.concat(df_list)

  df = df.reset_index(drop=True)

  df['current_team_nm'] = df['team_nm'].apply(get_current_team_name)

  return df

# Creating Teams Table

## 1. Creating team url df

In [6]:
teams_url = "https://www.pro-football-reference.com/teams/"

driver = get_webdriver()

# go to teams url
driver.get(teams_url)

delay = 30
active_teams_div = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.ID, 'all_teams_active')))

a_tags = active_teams_div.find_elements(By.TAG_NAME, 'a')

data = []
for tag in a_tags:
  link = tag.get_attribute('href')

  if link is not None and "teams" in link:
    team_name = tag.text
    data.append((team_name, link))

team_url_df = pd.DataFrame(data, columns=['team_nm', 'team_url'])

## 2. Creating Team df

In [28]:
teams_url = "https://www.pro-football-reference.com/teams/"

driver = get_webdriver()

# go to teams url
driver.get(teams_url)

delay = 30
active_teams_div = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.ID, 'all_teams_active')))

# Get the HTML of the table
table_html = active_teams_div.get_attribute('outerHTML')

# Use Pandas to read the HTML table
df = pd.read_html(table_html)[0]

# remove any teams that were disbanded before 2013
year_cutoff = 2013

# if the dataframe has Multiindex columns, flatten this multiindex
if df.columns.nlevels > 1:
  # renaming unnamed columns
  for i, columns in enumerate(df.columns.levels):
    columns_new = columns.tolist()
    for j, row in enumerate(columns_new):
        if "Unnamed: " in row:
            columns_new[j] = ""
    df = df.rename(columns=dict(zip(columns.tolist(), columns_new)), level=i)

  # flattening multiindex columns
  df.columns = ['_'.join(col).strip() for col in df.columns.values]
  df.columns = [transform_col(i[1:]) if i[0]=="_" else transform_col(i) for i in df.columns]
else:
  df.columns = [transform_col(i) for i in df.columns]

df = transform_dataframe(df)

# merge with team url df

print(f"team_df size before: {df.shape[0]}")

team_df = df.merge(team_url_df, left_on='current_team_nm', right_on='team_nm', how='left')

team_df = team_df.drop(['team_nm_x'], axis=1).rename(columns={'team_nm_y': 'team_nm'})

print(f"team_df size after: {team_df.shape[0]}")

display(team_df)

team_df size before: 37
team_df size after: 37


,first_year,last_year,games_won,games_lost,games_tied,win_loss_pct,years_in_playoffs,playoff_wins,playoff_losses,playoff_win_loss_pct,super_bowls_wins,conf_champ_wins,div_champ_wins,current_team_nm,team_nm,team_url
0,1994.0,2024.0,203.0,281.0,2.0,0.420,6.0,6.0,6.0,NaN,0.0,1.0,3.0,Arizona Cardinals,Arizona Cardinals,https://www.pro-football-reference.com/teams/crd/
1,1966.0,2024.0,391.0,505.0,6.0,0.437,14.0,10.0,14.0,0.417,0.0,2.0,7.0,Atlanta Falcons,Atlanta Falcons,https://www.pro-football-reference.com/teams/atl/
2,1996.0,2024.0,257.0,196.0,1.0,0.567,15.0,17.0,13.0,0.567,2.0,2.0,7.0,Baltimore Ravens,Baltimore Ravens,https://www.pro-football-reference.com/teams/rav/
3,1960.0,2024.0,475.0,501.0,8.0,0.487,23.0,19.0,21.0,0.475,0.0,4.0,15.0,Buffalo Bills,Buffalo Bills,https://www.pro-football-reference.com/teams/buf/
4,1995.0,2024.0,215.0,254.0,1.0,0.459,8.0,9.0,8.0,0.529,0.0,2.0,6.0,Carolina Panthers,Carolina Panthers,https://www.pro-football-reference.com/teams/car/
5,1922.0,2024.0,775.0,634.0,39.0,0.550,27.0,17.0,20.0,NaN,1.0,2.0,23.0,Chicago Bears,Chicago Bears,https://www.pro-football-reference.com/teams/chi/
6,1968.0,2024.0,394.0,473.0,5.0,0.455,16.0,10.0,16.0,0.385,0.0,3.0,12.0,Cincinnati Bengals,Cincinnati Bengals,https://www.pro-football-reference.com/teams/cin/
7,1946.0,2024.0,560.0,536.0,14.0,0.511,30.0,17.0,22.0,0.436,0.0,0.0,23.0,Cleveland Browns,Cleveland Browns,https://www.pro-football-reference.com/teams/cle/
8,1960.0,2024.0,563.0,415.0,6.0,0.576,36.0,36.0,31.0,0.537,5.0,8.0,26.0,Dallas Cowboys,Dallas Cowboys,https://www.pro-football-reference.com/teams/dal/
9,1960.0,2024.0,509.0,467.0,10.0,0.521,22.0,23.0,19.0,0.548,3.0,8.0,15.0,Denver Broncos,Denver Broncos,https://www.pro-football-reference.com/teams/den/


# Writing team_df to file

In [29]:
team_df.to_csv("/content/drive/MyDrive/SIADS_696/Data/teams.csv", index=False)

# Creating Roster Table

In [8]:
def get_roster_urls_yearly(df):
  for i in range(2013, 2024):
    df[f"{i}_roster_url"] = df['team_url'].apply(lambda x: x + f"{i}_roster.htm")

  return df

def write_to_file(obj, filename):
  with open(filename, "a+") as f:
    # writing json obj to file
    f.write(obj)

In [13]:
team_df = get_roster_urls_yearly(team_df)
delay = 30

save_file = "/content/drive/MyDrive/SIADS_696/team_rosters_data.txt"

seen_team_years = []

try:
  with open(save_file, "r+") as f:
    lines = f.readlines()

    for line in lines:
      data = json.loads(line)
      team_name = data['team']
      year = data['year']

      seen_team_years.append((team_name, year))
except:
  pass

print(f"seen_team_years:\n{seen_team_years}")

driver = get_webdriver()

for ind in team_df.index:
  team_name = team_df.loc[ind, 'current_team_nm']

  print(f"Team Name: {team_name}")

  for i in range(2013, 2024):
    if (team_name, i) in seen_team_years:
      continue

    roster_url = team_df.loc[ind, f'{i}_roster_url']

    print(f"Roster URL: {roster_url}")

    # go to roster url
    driver.get(roster_url)

    #######find all_starter div#######
    all_starter_div = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.ID, 'all_starters')))

    # Get the HTML of the table
    table_html = all_starter_div.get_attribute('outerHTML')

    # Use Pandas to read the HTML table
    starters_df = pd.read_html(table_html)[0]
    starters_df = starters_df[~(starters_df['Player'].str.lower().isin(['offensive starters', 'defensive starters']))]
    starters_df['Player'] = starters_df['Player'].str.replace("+", "").str.replace("*", "")
    starters_df['is_starter'] = True
    starters_df = starters_df.drop_duplicates()

    starters_len = starters_df.shape[0]

    #######find all_roster div#######
    all_roster_div = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.ID, 'all_roster')))

    # Get the HTML of the table
    table_html = all_roster_div.get_attribute('outerHTML')

    # Use Pandas to read the HTML table
    roster_df = pd.read_html(table_html)[0]

    delete_indices = []
    for ind1 in roster_df.index:
      try:
        row_list = roster_df.loc[ind1, :].values.flatten().tolist()[:5]
        if row_list == roster_df.columns.tolist()[:5]:
          delete_indices.append(ind1)
      except:
        continue

    roster_df = roster_df[~(roster_df['Player'].str.lower()=='team total')]

    roster_df = roster_df.drop(delete_indices, axis=0).drop_duplicates()

    roster_len = roster_df.shape[0]

    merged_df = pd.merge(starters_df[['Player', 'is_starter']], roster_df, on='Player', how='outer')
    merged_df['is_starter'] = merged_df['is_starter'].fillna(False)

    if roster_len != merged_df.shape[0]:
      print(f"starters_df.shape:\n{starters_df}")
      print(f"roster_df.shape:\n{roster_df}")
      print(f"merge().shape:\n{merged_df.shape}")

      continue

    data_dict = {
        'team': team_name,
        'year': i,
        'link': roster_url,
        'roster': merged_df.to_json(),
    }

    save_json = json.dumps(data_dict)+"\n"

    write_to_file(save_json, "/content/drive/MyDrive/SIADS_696/team_rosters_data.txt")


seen_team_years:
[('Arizona Cardinals', 2013), ('Arizona Cardinals', 2014), ('Arizona Cardinals', 2015), ('Arizona Cardinals', 2016), ('Arizona Cardinals', 2017), ('Arizona Cardinals', 2018), ('Arizona Cardinals', 2019), ('Arizona Cardinals', 2020), ('Arizona Cardinals', 2021), ('Arizona Cardinals', 2022), ('Arizona Cardinals', 2023), ('Atlanta Falcons', 2013), ('Atlanta Falcons', 2014), ('Atlanta Falcons', 2015), ('Atlanta Falcons', 2016), ('Atlanta Falcons', 2017), ('Atlanta Falcons', 2018), ('Atlanta Falcons', 2019), ('Atlanta Falcons', 2020), ('Atlanta Falcons', 2021), ('Atlanta Falcons', 2022), ('Atlanta Falcons', 2023), ('Baltimore Ravens', 2013), ('Baltimore Ravens', 2014), ('Baltimore Ravens', 2015), ('Baltimore Ravens', 2016), ('Baltimore Ravens', 2017), ('Baltimore Ravens', 2018), ('Baltimore Ravens', 2019), ('Baltimore Ravens', 2020), ('Baltimore Ravens', 2021), ('Baltimore Ravens', 2022), ('Baltimore Ravens', 2023), ('Buffalo Bills', 2013), ('Buffalo Bills', 2014), ('Buffal

Get JSON Objects From "/content/drive/MyDrive/SIADS_696/team_rosters_data.txt"

In [26]:
column_name_map = {
  'AV': "approx_value",
  'Age': "age",
  'BirthDate': "birth_date",
  'College/Univ': "college_univ",
  'Drafted (tm/rnd/yr)': "draft_info",
  'G': "games_played",
  'GS': "games_started",
  'Ht': "height",
  'No.': "number",
  'Player': "player_name",
  'Pos': "position",
  'Salary': "salary",
  'Wt': "weight",
  'Yrs': "years_played",
}

roster_dfs = []
with open("/content/drive/MyDrive/SIADS_696/team_rosters_data.txt", "r") as f:
  column_set = set()
  lines = f.readlines()
  for i, line in enumerate(lines):
    data = json.loads(line)

    link = data['link']
    team = data['team']
    roster_year = data['year']
    roster_df = pd.read_json(data['roster'])

    roster_df['team'] = team
    roster_df['roster_year'] = roster_year
    roster_df['link'] = link

    roster_df = roster_df.rename(columns=column_name_map)

    if i == 0:
      print(roster_df)

    roster_dfs.append(roster_df)

  all_rosters_df = pd.concat(roster_dfs)

all_rosters_df

           player_name  is_starter  number  age position  games_played  \
0        Carson Palmer        True     3.0   34       QB            16   
1   Rashard Mendenhall        True    28.0   26       RB            15   
2     Larry Fitzgerald        True    11.0   30       WR            16   
3        Michael Floyd        True    15.0   24       WR            16   
4             Jim Dray        True    81.0   27       TE            16   
5          Rob Housler        True    84.0   25       TE            13   
6       Bradley Sowell        True    79.0   24       LT            16   
7       Daryn Colledge        True    71.0   31       LG            16   
8        Lyle Sendlein        True    63.0   29        C            16   
9         Paul Fanaika        True    74.0   27       RG            16   
10        Eric Winston        True    73.0   30       RT            16   
11     Calais Campbell        True    93.0   27      LDE            16   
12        Dan Williams        True    

,player_name,is_starter,number,age,position,games_played,games_started,weight,height,college_univ,birth_date,years_played,approx_value,draft_info,team,roster_year,link,salary
0,Carson Palmer,True,3.0,34,QB,16,16.0,235.0,6-5,USC,12/27/1979,9,11.0,Cincinnati Bengals / 1st / 1st pick / 2003,Arizona Cardinals,2013,https://www.pro-football-reference.com/teams/c...,NaN
1,Rashard Mendenhall,True,28.0,26,RB,15,15.0,210.0,5-11,Illinois,6/19/1987,5,3.0,Pittsburgh Steelers / 1st / 23rd pick / 2008,Arizona Cardinals,2013,https://www.pro-football-reference.com/teams/c...,NaN
2,Larry Fitzgerald,True,11.0,30,WR,16,16.0,218.0,6-3,Pittsburgh,8/31/1983,9,7.0,Arizona Cardinals / 1st / 3rd pick / 2004,Arizona Cardinals,2013,https://www.pro-football-reference.com/teams/c...,NaN
3,Michael Floyd,True,15.0,24,WR,16,16.0,220.0,6-3,Notre Dame,11/27/1989,1,8.0,Arizona Cardinals / 1st / 13th pick / 2012,Arizona Cardinals,2013,https://www.pro-football-reference.com/teams/c...,NaN
4,Jim Dray,True,81.0,27,TE,16,15.0,253.0,6-5,Stanford,12/31/1986,3,2.0,Arizona Cardinals / 7th / 233rd pick / 2010,Arizona Cardinals,2013,https://www.pro-football-reference.com/teams/c...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,Rashad Weaver,False,99.0,24,DE,2,0.0,259.0,6-4,Pittsburgh,11/10/1997,Rook,0.0,Tennessee Titans / 4th / 135th pick / 2021,Tennessee Titans,2021,https://www.pro-football-reference.com/teams/o...,NaN
87,Nick Westbrook-Ikhine,False,15.0,24,WR,16,7.0,211.0,6-2,Indiana,3/21/1997,1,5.0,None,Tennessee Titans,2021,https://www.pro-football-reference.com/teams/o...,NaN
88,Jordan Wilkins,False,33.0,27,RB,1,0.0,216.0,6-1,Mississippi,7/18/1994,3,0.0,Indianapolis Colts / 5th / 169th pick / 2018,Tennessee Titans,2021,https://www.pro-football-reference.com/teams/o...,NaN
89,Avery Williamson,False,53.0,29,LB,2,0.0,246.0,6-1,Kentucky,3/9/1992,7,0.0,Tennessee Titans / 5th / 151st pick / 2014,Tennessee Titans,2021,https://www.pro-football-reference.com/teams/o...,NaN


# Write all_rosters_df to file

In [27]:
all_rosters_df.to_csv("/content/drive/MyDrive/SIADS_696/Data/rosters.csv", index=False)